In [1]:
# This notebook is a test of several properties that are needed 
# to stand up an end to end pool of online learners
from vowpalwabbit import pyvw

In [2]:
# Property 1: Inital test to verify that more than one vw can be used
# and that they do not share model memory

first_vw = pyvw.vw(quiet=True)
second_vw = pyvw.vw(quiet=True)
print(f"first vw has {hex(id(first_vw))} and second vw has {hex(id(second_vw))}")
id(first_vw) == id(second_vw), "First and second instances point to the same memory!"

# But they don't seem to share model memory? Note how even the examples are in scope of
# the instance
ex = first_vw.example('1 | a b c')
first_vw.learn(ex)

first_predict = first_vw.predict(ex)
second_predict = second_vw.predict(ex)

print(f"first model predicted {first_predict}, second predicted  {second_predict}")
assert first_predict != second_predict, "Models seems equivalent because same prediction was given"

first vw has 0x7fd910737630 and second vw has 0x7fd910731b30
first model predicted 0.632030725479126, second predicted  0.0


In [3]:
# Property 2: Referencing a config file, create a pool of learners
# The pool should be threadsafe (deduction from locked_dict library)
from dynaconf import settings
import locked_dict.locked_dict as locked_dict
from coolname import generate_slug

pool_of_learners = locked_dict.LockedDict()

a_key = None
for _ in range(settings.NUMBER_OF_LEARNERS):
    name = generate_slug(2)
    print(f"\t ...adding {name}")
    pool_of_learners[name] = pyvw.vw(quiet=True)
    a_key = name
    
assert type(pool_of_learners[a_key]) == type(pyvw.vw(quiet=True)), "Pool contains different types than vw!"

	 ...adding singing-swine
	 ...adding hissing-yak


In [17]:
# Property 3: ImageNet can recieve and output features that a learner can accept

# class code attribution @sansi95 (https://github.com/robinsonkwame/kente-cloth-authentication/src/features/feature_processor.py)
# modified
from abc import ABC, abstractmethod
import numpy as np
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from dynaconf import settings # because we frequently update the file 
import PIL

class FeatureProcessor(ABC):
    def __init__(self,
                 batch_size,
                 flattened_size,
                 feature_file_format
                ):
        super().__init__()
        self.batch_size = batch_size
        self.flattened_size = flattened_size
        self.feature_file_format = feature_file_format

    @staticmethod
    def create(
        feature_processor_name,
        batch_size,
        feature_file_format
    ):
        if feature_processor_name == "MobileNet":
            flattened_size = 7 * 7 * 1280
            return MobileNetFeatureProcessor(batch_size, flattened_size, feature_file_format)

    def initialize_output_processor(self, labels, feature_file_path):
        if self.feature_file_format == "npy":
            self.output_processor = NpyOutput(labels,
            self.flattened_size, self.batch_size, feature_file_path)
        elif self.feature_file_format == "csv":
            self.output_processor = CsvOutput(labels,
            self.batch_size, feature_file_path)

    @abstractmethod
    def process_image(self):
        pass

    @abstractmethod
    def create_features(self):
        pass

class MobileNetFeatureProcessor(FeatureProcessor):
    def __init__(self, batch_size, flattened_size, feature_file_format):
        super().__init__(batch_size, flattened_size, feature_file_format)
        self.model = MobileNetV2(weights="imagenet",
                                 include_top=False, 
                                 input_shape=(224, 224, 3)
                    )
        self.name = "mobile"

    def process_image(self, image_path):
        image = load_img(
            image_path,
            target_size=(224, 224)
        )
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = imagenet_utils.preprocess_input(image)
        return image

    def create_features(self, batch_images):
        features = self.model.predict(
            batch_images,
            batch_size= self.batch_size
        )
        features = features.reshape(
            (features.shape[0], self.flattened_size)
        )
        return features

def construct_vw_example(label, image_array):
    return f"{label} |" + np.array2string(
        image_array,
        precision=4,
        separator=' ',
        suppress_small=True
    )

    
the_feature_processor = FeatureProcessor.create(
        feature_processor_name=settings.FEATURE_PROCESSOR_NAME,
        batch_size=settings.BATCH_SIZE,
        feature_file_format=None
)

the_image_features = the_feature_processor.create_features(
    the_feature_processor.process_image('./real_miami_a_25.jpg')
)
assert the_image_features.shape[1] == 7 * 7 * 1280, "feature vector is not the expected length"

TRUE = 1
image_feature_ex = pool_of_learners[a_key].example(
    construct_vw_example(TRUE, the_image_features)
)
pool_of_learners[a_key].learn(image_feature_ex)

decision = pool_of_learners[a_key].predict(image_feature_ex)
assert decision == TRUE, "The learned example was predicted to be something other than the example!"
print(f"the decision was {decision}")

the decision was 1.0


Object `pool_of_learners[a_key]` not found.
